In [8]:
import pandas as pd
from sklearn.base import TransformerMixin
from numpy import ravel
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline, FeatureUnion

data_path = 'data/'

In [9]:
# contains all methods and classes for FeatureUnion
## generic read_csv

#example
class Regular(TransformerMixin):
    #return itself
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        print(X.shape)
        return X

#example
class LoadCSV(TransformerMixin):
    #return itself
    def __init__(self, filename):
        self.filename = filename
    
    def fit(self, x, y=None):
        return self
    
    def transform(self, X):
        # we assume the first CSV entry is always the ID
        return pd.read_csv(self.filename, index_col = 0)

# reg = Regular()
# feature = LoadCSV('file.csv')

In [12]:
# individual import statements for every feature, comment these out if needed
#LoadCSV('data/train_fail_date_score.csv').fit_transform()
union = FeatureUnion([('first', LoadCSV('data/train_fail_date_score.csv'))])
union.fit_transform(1)


array([[  5.],
       [ 10.],
       [  5.],
       ..., 
       [  5.],
       [ 26.],
       [ 29.]])

In [ ]:
# one featureunion for everything if possible
## always new code cell for testing pipelines